In [1]:
import os, sys, time, importlib, osmium, logging, pyproj
import osmnx

import geopandas as gpd
import pandas as pd
import networkx as nx
import numpy as np
import shapely.wkb as wkblib

from functools import partial
from shapely.ops import transform
from shapely.geometry import Point, LineString

# import current version of GOSTnets
sys.path.insert(0,"../")

import GOSTnets as gn
import GOSTnets.load_osm as losm
import GOSTnets.load_osmium as losmium
import GOSTnets.mapbox_traffic as mb
import GOSTnets.network_clean

In [2]:
# Test osmium load to generate G
osm_file = r"C:\temp\dhaka-latest.osm.pbf"
mapbox_traffic_file = "C:/temp/1320222-Asia-Dhaka.csv"

# Test various attempts to read in G object

In [ ]:
G_osm = losm.OSM_to_network(osm_file)
G_regular = G_osm.initialReadIn()

In [ ]:
print(G_regular.number_of_nodes())
print(G_regular.number_of_edges())
print(gn.example_edge(G_regular))
print(gn.example_node(G_regular))

In [ ]:
importlib.reload(losmium)
h = losmium.HighwayExtractor(False)
h.apply_file(osm_file, locations=True)

In [ ]:
nodes_df = pd.DataFrame(h.nodes, columns = ["osm_id", "geometry", "x", "y"])

In [ ]:
importlib.reload(losmium)
G = losmium.create_G(osm_file, verbose=False, project='', densify = False)
#G = nx.convert_node_labels_to_integers(G)

In [ ]:
print(G.number_of_nodes())
print(G.number_of_edges())
print(gn.example_edge(G))
print(gn.example_node(G))

In [ ]:
#Extract largest subgraph from regular load
list_of_subgraphs = list(nx.strongly_connected_component_subgraphs(G_regular))
print(len(list_of_subgraphs))
max_graph = None
max_edges = 0
for i in list_of_subgraphs:
    if i.number_of_edges() > 0:
        print(i.number_of_edges())
    if i.number_of_edges() > max_edges:
        max_edges = i.number_of_edges()
        max_graph = i
print(max_edges)
G_max = max_graph

In [ ]:
#Extract largest subgraph from regular load
list_of_subgraphs = list(nx.strongly_connected_component_subgraphs(G))
print(len(list_of_subgraphs))
max_graph = None
max_edges = 0
for i in list_of_subgraphs:   
    if i.number_of_edges() > 2:
        print(i.number_of_edges())
    if i.number_of_edges() > max_edges:
        max_edges = i.number_of_edges()
        max_graph = i
print(max_edges)
G_osmium = max_graph

In [ ]:
print(list(G_max.nodes(data=True))[0])
print(list(G_max.edges(data=True))[0])

In [ ]:
# Write to file
gn.edge_gdf_from_graph(G_max).to_csv("C:/temp/G_max_edges.csv")

In [ ]:
print(list(G_osmium.nodes(data=True))[0])
print(list(G_osmium.edges(data=True))[0])

In [ ]:
gn.edge_gdf_from_graph(G_osmium, geometry_tag='Wkt').to_csv("C:/temp/G_osmium_edges.csv")

In [ ]:
print(G.number_of_edges())
print(G.number_of_nodes())

In [ ]:
%%time
G2 = losm.create_G(osm_file, verbose=False, project=3106, densify = False)

In [ ]:
print(G2.number_of_edges())
print(G2.number_of_nodes())

In [ ]:
%%time
G3 = losmium.create_G(osm_file, verbose=False, project='', densify = True)

In [ ]:
print(G3.number_of_edges())
print(G3.number_of_nodes())

In [ ]:
%%time
G4 = losm.create_G(osm_file, verbose=False, project=3106, densify = True)

In [ ]:
print(G4.number_of_edges())
print(G4.number_of_nodes())

# Read in and process mapbox data

In [ ]:
# Test processing of mapbox traffic files
mb_traffic = mb.simplify_mapbox_csv(mapbox_traffic_file)

In [ ]:
list_of_subgraphs = list(nx.strongly_connected_component_subgraphs(G))
print(len(list_of_subgraphs))
max_graph = None
max_edges = 0
for i in list_of_subgraphs:
    print(i.number_of_edges())
    if i.number_of_edges() > max_edges:
        max_edges = i.number_of_edges()
        max_graph = i

In [ ]:
importlib.reload(GOSTnets.network_clean)
G_clean = GOSTnets.network_clean.eliminate_small_networks(G)

In [ ]:
print(G.number_of_edges())
print(G_clean.number_of_edges())

In [ ]:
importlib.reload(mb)

xx = mb.attach_traffic_data_ways(G, mb_traffic)

In [ ]:
# Generate reference to the edges and node datasets from the network
cur_highways = gn.edge_gdf_from_graph(G, geometry_tag='Wkt')

In [ ]:
cur_highways

In [ ]:
xx.head()

In [ ]:
cur_highways['mb_min'] = 0
cur_highways['mb_avg'] = 0
cur_highways['mb_max'] = 0

for idx, row in cur_highways.iterrows():
    nodes = row['osm_nodes']
    min_speed = []
    max_speed = []
    avg_speed = []
    for nIdx in range(0, len(nodes) - 1):
        st_node = nodes[nIdx]
        end_node = nodes[nIdx + 1]
        try:
            cur_speed = mb_traffic.loc[(mb_traffic['FROM_NODE'] == st_node) & 
                                       (mb_traffic['TO_NODE'] == end_node),].iloc[0]
            min_speed.append(cur_speed['min_speed'])
            max_speed.append(cur_speed['max_speed'])
            avg_speed.append(cur_speed['mean_speed'])
            all_speeds.append(cur_speed)
        except:
            pass
    cur_highways.loc[idx, 'min_speed'] = np.mean(min_speed)
    cur_highways.loc[idx, 'max_speed'] = np.mean(max_speed)
    cur_highways.loc[idx, 'avg_speed'] = np.mean(avg_speed)


In [ ]:
cur_highways['min_speed'].fillna(0, inplace=True)
cur_highways['max_speed'].fillna(0, inplace=True)
cur_highways['avg_speed'].fillna(0, inplace=True)

In [ ]:
cur_dict = gn.gost_speed_dict
cur_dict['service'] = 10
cur_dict['track'] = 5
cur_highways['speed'] = cur_highways['infra_type']
cur_highways['speed'].replace(gn.gost_speed_dict, inplace=True)
cur_highways['mb_speed'].fillna(0, inplace=True)
cur_highways['diff'] = cur_highways['speed'] - cur_highways['mb_speed']

In [ ]:
cur_highways.to_csv("C:/temp/highways2.csv")

In [ ]:
cur_highways.head(20)

In [ ]:
cur_highways.shape